In [2]:
# 初始化spark环境
import findspark
findspark.init()
import sys
print(sys.version_info)

sys.version_info(major=2, minor=7, micro=14, releaselevel='final', serial=0)


In [3]:
from pyspark import SparkContext
from pyspark import SparkConf

In [5]:
# 创建Spark对应此应用程序的配置，必须设置应用名称，必须设置运行模式， 
# local[*] 本地测试用的模式，中括号内指的线程数，*代表满负载执行
# local 等价于 local[1]
conf = SparkConf().setAppName("chihweihsu").setMaster("local[*]") 
#创建Spark上下文环境，接收的参数通常是SparkConf
sc = SparkContext(conf=conf)

In [11]:
# data 在Python解释器中
data = ["I love huzhiqin", "I love liuyifei", "I love liangyongqi"]
# 把 data 并行化变成了RDD（弹性分布式数据集）
#RDD分布式存在Spark集群中
rdd = sc.parallelize(data)
print rdd

ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:423


In [13]:
rdd.map(lambda sen: sen.split()).collect()

[['I', 'love', 'huzhiqin'],
 ['I', 'love', 'liuyifei'],
 ['I', 'love', 'liangyongqi']]

In [19]:
# flatmap = flat（集合中元素扁平化） + map
# 先map，再flat压扁
rdd.flatMap(lambda sen: sen.split()).collect()

['I', 'love', 'huzhiqin', 'I', 'love', 'liuyifei', 'I', 'love', 'liangyongqi']

In [17]:
rdd.flatMap(lambda sen: sen.split()).map(lambda word: (word, 1)).collect()

[('I', 1),
 ('love', 1),
 ('huzhiqin', 1),
 ('I', 1),
 ('love', 1),
 ('liuyifei', 1),
 ('I', 1),
 ('love', 1),
 ('liangyongqi', 1)]

In [26]:
rdd.flatMap(lambda sen: sen.split()).map(lambda word: (word, 1)).groupByKey().collect()

[('I', <pyspark.resultiterable.ResultIterable at 0x8cce320>),
 ('love', <pyspark.resultiterable.ResultIterable at 0x8cce438>),
 ('liuyifei', <pyspark.resultiterable.ResultIterable at 0x8cce128>),
 ('huzhiqin', <pyspark.resultiterable.ResultIterable at 0x8ccebe0>),
 ('liangyongqi', <pyspark.resultiterable.ResultIterable at 0x8cce828>)]

In [27]:
rdd.flatMap(lambda sen: sen.split()).map(lambda word: (word, 1)).groupByKey().mapValues(list).collect()

[('I', [1, 1, 1]),
 ('love', [1, 1, 1]),
 ('liuyifei', [1]),
 ('huzhiqin', [1]),
 ('liangyongqi', [1])]

In [28]:
# reduceByKey = reduce + groupByKey
# 先执行groupByKey（按照键分组）， 再执行reduce（按照匿名函数聚合运算）
# collect（将运算结果从Spark端拉回Python解释器中）
rdd.flatMap(lambda sen: sen.split()).map(lambda word: (word, 1)).reduceByKey(lambda x,y: x+y).collect()

[('I', 3), ('love', 3), ('liuyifei', 1), ('huzhiqin', 1), ('liangyongqi', 1)]